In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cdist

### Carrega e Normaliza os Dados

In [2]:
tracks = pd.read_csv('../tracks/tracks.csv')
tracks

,id,name,duration_ms,artists,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature
0,spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,52213,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War...",t,0,0.004530,0.324,0.971,0.012800,0,0.3340,-3.168,1,0.0548,95.721,4
1,spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],88888,"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}",f,0,0.833000,0.545,0.495,0.629000,11,0.3860,-13.538,1,0.1020,62.631,4
2,spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,564720,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2...",f,0,0.033200,0.695,0.592,0.892000,0,0.5790,-9.588,1,0.0345,125.008,4
3,spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,245356,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}",t,0,0.000440,0.450,0.629,0.895000,7,0.1080,-7.059,1,0.1460,193.228,3
4,spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,315773,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}",f,0,0.000927,0.846,0.707,0.903000,6,0.0666,-5.949,0,0.1270,127.970,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261611,spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,294333,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}",f,2,0.109000,0.538,0.609,0.000006,5,0.0948,-7.392,1,0.0275,87.960,4
2261612,spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,202733,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}",f,0,0.698000,0.339,0.325,0.000005,2,0.5550,-11.445,1,0.0252,91.286,4
2261613,spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,293546,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}",f,0,0.018600,0.482,0.443,0.000345,4,0.1100,-8.622,1,0.0276,144.962,4
2261614,spotify:track:7LdoOfaFSBX9Cy2R3NGcAx,Reverse Skydiving (feat. Anabel Englund),329065,"{""(4aKZ8rfdsQeR7YSskFu9V3,\""Hot Natured\"")"",""(...",f,46,0.006410,0.915,0.461,0.059500,11,0.0771,-7.965,0,0.0505,118.013,4


In [18]:
# Músicas que vão simular as músicas do usuário a serem recomendadas
classical_music_i = tracks.index[tracks['name'] == 'Symphony No.5 in C minor, Op.67: 1. Allegro con brio'][0]
rap_music = tracks.index[tracks['name'] == 'Yoshi City'][0]
anime_music = tracks.index[tracks['name'] == 'GO!!!'][0]
kaio_music = tracks.index[tracks['name'] == 'Wrist Cry'][0]
kaio_music2 = tracks.index[tracks['name'] == 'Someday'][0]

KeyError: 'name'

In [5]:
# guarda o id e nome
music_ids = tracks[['id', 'name']]
# remove colunas que não serão utilizadas
tracks.drop(['id', 'name', 'artists', 'explicit', 'time_signature', 'mode'], axis=1, inplace=True) 

In [6]:
tracks

,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
0,52213,0,0.004530,0.324,0.971,0.012800,0,0.3340,-3.168,0.0548,95.721
1,88888,0,0.833000,0.545,0.495,0.629000,11,0.3860,-13.538,0.1020,62.631
2,564720,0,0.033200,0.695,0.592,0.892000,0,0.5790,-9.588,0.0345,125.008
3,245356,0,0.000440,0.450,0.629,0.895000,7,0.1080,-7.059,0.1460,193.228
4,315773,0,0.000927,0.846,0.707,0.903000,6,0.0666,-5.949,0.1270,127.970
...,...,...,...,...,...,...,...,...,...,...,...
2261611,294333,2,0.109000,0.538,0.609,0.000006,5,0.0948,-7.392,0.0275,87.960
2261612,202733,0,0.698000,0.339,0.325,0.000005,2,0.5550,-11.445,0.0252,91.286
2261613,293546,0,0.018600,0.482,0.443,0.000345,4,0.1100,-8.622,0.0276,144.962
2261614,329065,46,0.006410,0.915,0.461,0.059500,11,0.0771,-7.965,0.0505,118.013


### Função de normalização usando fórmula do MinMaxScaler do SKLearn 
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [7]:
# função que recebe um pandas Series e a escala sem copiar memória
def scaler(X, min_=None, max_=None):
    if not min_:
        min_ = X.min()
    if not max_:
        max_ = X.max()
    return X.map(lambda x: (x - min_) / (max_ - min_))

### Normalização dos dados que não estão normalizados

In [8]:
tracks['duration_ms'] = scaler(tracks['duration_ms'])
tracks['popularity'] = scaler(tracks['popularity'], min_=0, max_=100)
tracks['key'] = scaler(tracks['key'])
tracks['loudness'] = scaler(tracks['loudness'], min_=-60, max_=0)
tracks['tempo'] = scaler(tracks['tempo'])

In [9]:
tracks

,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
0,0.002590,0.00,0.004530,0.324,0.971,0.012800,0.000000,0.3340,0.875375,0.0548,0.382904
1,0.004409,0.00,0.833000,0.545,0.495,0.629000,1.000000,0.3860,0.715648,0.1020,0.250537
2,0.028013,0.00,0.033200,0.695,0.592,0.892000,0.000000,0.5790,0.776489,0.0345,0.500058
3,0.012171,0.00,0.000440,0.450,0.629,0.895000,0.636364,0.1080,0.815443,0.1460,0.772952
4,0.015664,0.00,0.000927,0.846,0.707,0.903000,0.545455,0.0666,0.832540,0.1270,0.511907
...,...,...,...,...,...,...,...,...,...,...,...
2261611,0.014601,0.02,0.109000,0.538,0.609,0.000006,0.454545,0.0948,0.810314,0.0275,0.351858
2261612,0.010057,0.00,0.698000,0.339,0.325,0.000005,0.181818,0.5550,0.747886,0.0252,0.365163
2261613,0.014561,0.00,0.018600,0.482,0.443,0.000345,0.363636,0.1100,0.791368,0.0276,0.579878
2261614,0.016323,0.46,0.006410,0.915,0.461,0.059500,1.000000,0.0771,0.801488,0.0505,0.472077


## Criando Modelo de Similaridade

In [10]:
class Similarity:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
    
    def recommend(self, user_tracks, n_tracks=5):
        
        mean_track = np.mean(user_tracks, axis=0)

        dists = cdist([mean_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks+1)[:n_tracks+1]
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=k_arg_dists)

        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])
        
        

In [11]:
sim = Similarity()
sim.fit(tracks, music_ids)
sim.recommend([tracks.iloc[0]])

,Distâncias,id,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
0,0.000000,spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,0.002590,0.00,0.004530,0.324,0.971,0.012800,0.0,0.334,0.875375,0.0548,0.382904
501312,0.044851,spotify:track:5EW0PGHJQ4wB1TWLGfaFWo,Smash Or Be Smashed,0.007887,0.00,0.000126,0.324,0.960,0.000919,0.0,0.365,0.869445,0.0762,0.398649
47077,0.048931,spotify:track:3jqBDCG0UliKiiBy49BEql,Tim Dorsey Writes Non-Fiction,0.006746,0.01,0.000055,0.294,0.938,0.000032,0.0,0.335,0.873003,0.0645,0.380848
1455595,0.048961,spotify:track:7iTMKu6hSvCD7jYg2XXW6h,Monkey On Your Back - Live,0.009394,0.00,0.000296,0.303,0.967,0.000371,0.0,0.318,0.870832,0.0733,0.349698
979952,0.059958,spotify:track:0ybC7e33YPDabKrnPPYupJ,People I Used To Know - Bonus Track,0.007855,0.01,0.006280,0.356,0.946,0.000006,0.0,0.325,0.857015,0.0552,0.418022
2221396,0.060820,spotify:track:7LRPbYCAHPCDg4qoIYDTJv,Erotic Neurotic,0.011037,0.00,0.001750,0.338,0.978,0.000473,0.0,0.284,0.867212,0.0613,0.407685


In [12]:
sim2 = Similarity()
sim2.fit(tracks, music_ids)
sim2.recommend([tracks.iloc[classical_music_i]])

,Distâncias,id,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
215695,0.000000,spotify:track:4Znak3UnsXjRYQDSK0oP6V,"Symphony No.5 in C minor, Op.67: 1. Allegro co...",0.021257,0.00,0.900,0.309,0.307,0.745,0.0,0.0929,0.764198,0.0443,0.402445
676349,0.083024,spotify:track:0ukGEGDhJzWbtSDUCuhgqa,Tomorrow Is My Dancing Day,0.010850,0.03,0.863,0.353,0.337,0.747,0.0,0.1120,0.730804,0.0310,0.398105
763275,0.096326,spotify:track:68DobenRkD7JT0IP9fX3sJ,Have Yourself a Merry Little Christmas,0.011655,0.00,0.886,0.290,0.296,0.713,0.0,0.1510,0.737089,0.0308,0.345950
665211,0.101696,spotify:track:1iJ88Pdx1n3RNuYwAJpQVl,Mkhtarat Wahab,0.013813,0.00,0.956,0.328,0.329,0.751,0.0,0.1190,0.786301,0.0367,0.473453
1988859,0.102900,spotify:track:60uhqRMd1I8TblpuESinU2,"Symphony No.5 in C minor, Op.67: 4. Allegro",0.024767,0.00,0.897,0.275,0.310,0.828,0.0,0.0739,0.785561,0.0332,0.362791
240573,0.103692,spotify:track:5JFvdPtX9hCvsWE32cndEu,Io vivo nella luna,0.008654,0.00,0.925,0.318,0.359,0.707,0.0,0.1180,0.755449,0.0384,0.331749


In [13]:
sim3 = Similarity()
sim3.fit(tracks, music_ids)
sim3.recommend([tracks.iloc[rap_music]])

,Distâncias,id,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
297206,0.000000,spotify:track:5Tou8lYopj9qCIFBFBomZ5,Yoshi City,0.011165,0.47,0.476,0.546,0.530,0.000000,0.090909,0.0955,0.784637,0.211,0.512863
1369986,0.086241,spotify:track:6TC0snyor5XOJzP6dGxRkn,On My Way,0.010064,0.45,0.480,0.584,0.469,0.000000,0.090909,0.0909,0.788041,0.169,0.520519
1731501,0.125333,spotify:track:00VWUa96Clte4uNEv963vn,Évite la,0.010661,0.42,0.442,0.603,0.538,0.000049,0.181818,0.1010,0.786347,0.232,0.513463
1159592,0.138520,spotify:track:26xvDUjGqtWGkiqXQJH2Dy,Somnífera,0.015886,0.42,0.481,0.552,0.493,0.000252,0.181818,0.1050,0.787471,0.163,0.580454
1479841,0.143725,spotify:track:4GzrlsibHiAfMqyWvSBvgh,Dos Locos,0.010226,0.53,0.435,0.601,0.603,0.000000,0.090909,0.0877,0.834974,0.258,0.559953
498675,0.147240,spotify:track:28d0FhfAskw8XgbKZRCAQY,The Wolf,0.013426,0.47,0.560,0.578,0.562,0.000012,0.181818,0.1520,0.803028,0.190,0.531152


In [14]:
sim4 = Similarity()
sim4.fit(tracks, music_ids)
sim4.recommend([tracks.iloc[anime_music]])

,Distâncias,id,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
906457,0.000000,spotify:track:0zFRHGYU1zP6q45qbils3z,GO!!!,0.011869,0.0,0.05080,0.604,0.978,0.000000,1.0,0.228,0.891595,0.0573,0.536584
1447665,0.054441,spotify:track:32eNGGyGv2gRsQfcwweMsf,Samir's Theme - Original Mix,0.011347,0.0,0.05880,0.649,0.971,0.000871,1.0,0.215,0.898695,0.0580,0.511991
1581042,0.059658,spotify:track:3OwbRWRebostjR3D09IKDV,For Reverend Green,0.019578,0.0,0.03960,0.635,0.993,0.000020,1.0,0.225,0.849237,0.0431,0.523043
451658,0.060158,spotify:track:1KxjkSBJTupOyp39oWJdch,Hands,0.012232,0.0,0.04020,0.617,0.946,0.003520,1.0,0.251,0.854258,0.0398,0.528111
1480392,0.060424,spotify:track:7hW0ePvGR0J4bOcNqOUEwM,Peace,0.016774,0.0,0.02950,0.604,0.984,0.000086,1.0,0.230,0.855552,0.0787,0.499486
389095,0.062549,spotify:track:7FGI4qUo3vyXHi4O7mRcs4,Protection (Demo),0.010416,0.0,0.00217,0.591,0.956,0.000000,1.0,0.235,0.876269,0.0802,0.527411


In [15]:
sim5 = Similarity()
sim5.fit(tracks, music_ids)
sim5.recommend([tracks.iloc[kaio_music]])

,Distâncias,id,name,duration_ms,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo
509178,0.000000,spotify:track:15TJBhID2NsMMp2etO0B03,Wrist Cry,0.011635,0.32,0.366,0.634,0.428,0.003270,0.727273,0.280,0.738906,0.0284,0.439895
1318949,0.112160,spotify:track:0oXj2IvCYO4UyIza4C4mSs,Blue Period - Acoustic,0.008807,0.27,0.318,0.699,0.431,0.000022,0.727273,0.237,0.741925,0.0251,0.399221
1844517,0.114308,spotify:track:2VFofCy8DixMed9fUwDULV,Baby,0.009189,0.27,0.383,0.650,0.451,0.000005,0.636364,0.278,0.743881,0.0372,0.473137
319934,0.118611,spotify:track:5sFjD0Zs5g4tSSCDbl9jzT,"Recuerdos ""Memorie""",0.016064,0.31,0.273,0.657,0.417,0.000009,0.727273,0.213,0.742695,0.0291,0.428254
2259369,0.125570,spotify:track:0XHRg1fJGPNgMLPsvolLj1,HAPPINESS - Acoustic,0.011060,0.41,0.365,0.650,0.454,0.000000,0.727273,0.297,0.803259,0.0287,0.392004
663636,0.125964,spotify:track:0f0tkH1v7YeyCiTiYE4tmY,Ojitos Negros,0.008832,0.32,0.483,0.622,0.419,0.000026,0.727273,0.270,0.777382,0.0387,0.424142
